In [1]:
import torch
import numpy as np
from torch_geometric.data import InMemoryDataset, Data
def load_raw():
    from sklearn.preprocessing import LabelBinarizer
    data = np.load("dataset/raw/data.npy")
    label = np.load("dataset/raw/label.npy")
    data = data[...,:3]
    one_hot = LabelBinarizer()
    label = one_hot.fit_transform(label)
    print(data.shape)
    print(label.shape)
    return data, label
CONNECTION = torch.tensor([[0,0,0,1,1,2,2,3,3,4,5,5,5,6,6,7,7,8,9,9,9,10,10,11,11,12,13,13,13,14,14,15,15,16,17,17,17,18,18,19,19,20],
                           [1,5,17,0,2,1,3,2,4,3,0,9,6,5,7,8,6,7,5,10,13,9,11,10,12,11,9,14,17,13,15,14,16,15,0,13,18,17,19,18,20,19]], dtype=torch.long)

#这里给出大家注释方便理解
class MyOwnDataset(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None):
        super().__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])
        print(self[0].x.shape)
    #返回数据集源文件名
    @property
    def raw_file_names(self):
        return ['./dataset/raw/data.npy', 'dataset/raw/label.npy']
    #返回process方法所需的保存文件名。你之后保存的数据集名字和列表里的一致
    @property
    def processed_file_names(self):
        return ['data.pt']
    #生成数据集所用的方法
    def process(self):
        data, label = load_raw()
        # Read data into huge `Data` list.
        data_list = []
        for i in range(data.shape[0]):
            
            pr_data = Data(x=torch.tensor(data[i], dtype=torch.float), edge_index=CONNECTION, y=torch.tensor([label[i]], dtype=float))
            data_list.append(pr_data)
        if self.pre_filter is not None:
            data_list = [data for data in data_list if self.pre_filter(data)]

        if self.pre_transform is not None:
            data_list = [self.pre_transform(data) for data in data_list]

        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])
dataset = MyOwnDataset("./dataset")

torch.Size([21, 3])


In [2]:
dataset = dataset.shuffle()
train_dataset = dataset[:240000]
test_dataset = dataset[240000:]

from torch_geometric.loader import DataLoader
BATCH = 16384
train_loader = DataLoader(train_dataset, batch_size=BATCH, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH, shuffle=True)

In [3]:

import torch.nn.functional as F
import torch.nn as nn
from torch_geometric.nn import GCNConv, global_mean_pool, Sequential
class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        self.net = Sequential(
            input_args = 'x, edge_index',
            modules=(
                (GCNConv(dataset.num_node_features, 64),'x, edge_index -> x'),
                nn.ReLU(True),
                (GCNConv(64, 128), 'x, edge_index -> x'),
                nn.ReLU(True),
                (GCNConv(128, 64), 'x, edge_index -> x'),
                (lambda x:global_mean_pool(x, torch.tensor([BATCH-1], dtype=torch.int64).to(device)), 'x-> x'),
                nn.Linear(64, dataset.num_classes),
            )
        )

    def forward(self, x, edge_index):
        x = self.net(x, edge_index)
        return x
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


model = GCN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()
    
    for data in train_loader:  # Iterate in batches over the training dataset.
        data.to(device)
        out = model(data.x, data.edge_index)  # Perform a single forward pass.
        loss = criterion(out, data.y)  # Compute the loss.
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()  # Clear gradients.

def test(loader):
    model.eval()
    
    correct = 0
    for data in loader:  # Iterate in batches over the training/test dataset.
        data.to(device)
        out = model(data.x, data.edge_index)  
        pred = out.argmax(dim=1)  # Use the class with highest probability.
        correct += int((pred == data.y.argmax(dim=1)).sum())  # Check against ground-truth labels.
    return correct / len(loader.dataset)  # Derive ratio of correct predictions.

for epoch in range(1, 10):
    train()
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

ValueError: Expected input batch_size (16384) to match target batch_size (10624).